<a href="https://colab.research.google.com/github/sowmen/Deepfake-Challenge-Kaggle/blob/master/LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
import os, sys, time
import math
import random
import cv2
from tqdm import tqdm_notebook as tqdm
import pickle 

%matplotlib inline
pd.set_option('max_colwidth', None)

import warnings
warnings.filterwarnings('ignore')

In [3]:
sample_df = pd.read_csv('/content/drive/My Drive/100_2.csv')

real_dir = [f for f in sample_df.video_path if sample_df.loc[sample_df.video_path == f].label.item() == 0]
fake_dir = [f for f in sample_df.video_path if sample_df.loc[sample_df.video_path == f].label.item() == 1]
print(real_dir[0:5])
print(fake_dir[0:5])

video_dir = []
for f in real_dir:
    video_dir.append((f,0))
for f in fake_dir:
    video_dir.append((f,1))

['/content/drive/My Drive/Dataset/dfdc_train_part_0/nweufafotd.mp4', '/content/drive/My Drive/Dataset/dfdc_train_part_0/kqlvggiqee.mp4', '/content/drive/My Drive/Dataset/dfdc_train_part_2/rbhecseopz.mp4', '/content/drive/My Drive/Dataset/dfdc_train_part_2/yrxljghbmn.mp4', '/content/drive/My Drive/Dataset/dfdc_train_part_2/iweugtdrpv.mp4']
['/content/drive/My Drive/Dataset/dfdc_train_part_0/vstmxsekmm.mp4', '/content/drive/My Drive/Dataset/dfdc_train_part_0/pzdfwdnmyx.mp4', '/content/drive/My Drive/Dataset/dfdc_train_part_1/beoobdybbw.mp4', '/content/drive/My Drive/Dataset/dfdc_train_part_2/rxhklivqeh.mp4', '/content/drive/My Drive/Dataset/dfdc_train_part_2/zsfpyhgxmg.mp4']


In [0]:
random.shuffle(video_dir)

# **Frame Seperation**

In [0]:
input_size = 224
frames_per_video = 20

def _read_frames_at_indices(path, capture, frame_idxs):
    try:
        frames = []
        idxs_read = []
        for frame_idx in range(frame_idxs[0], frame_idxs[-1] + 1):
            # Get the next frame, but don't decode if we're not using it.
            ret = capture.grab()
            if not ret:
                print("Error grabbing frame %d from movie %s" % (frame_idx, path.split('/',4)[-1]))
                break

            # Need to look at this frame?
            current = len(idxs_read)
            if frame_idx == frame_idxs[current]:
                ret, frame = capture.retrieve()
                if not ret or frame is None:
                    print("Error retrieving frame %d from movie %s" % (frame_idx, path.split('/',4)[-1]))
                    break

                frame = _postprocess_frame(frame)
                frames.append(frame)
                idxs_read.append(frame_idx)

        if len(frames) > 0:
            return frames
        print("No frames read from movie %s" % path)
        return None
    except:
        print("Exception while reading movie %s" % path.split('/',4)[-1])
        return None 

def _postprocess_frame(frame, insets=(0,0)):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    if insets[0] > 0:
        W = frame.shape[1]
        p = int(W * insets[0])
        frame = frame[:, p:-p, :]

    if insets[1] > 0:
        H = frame.shape[1]
        q = int(H * insets[1])
        frame = frame[q:-q, :, :]

    return frame 

def read_frames(path, num_frames=frames_per_video, jitter=0, seed=None):
    """Reads frames that are always evenly spaced throughout the video.

        Arguments:
            path: the video file
            num_frames: how many frames to read, -1 means the entire video
                (warning: this will take up a lot of memory!)
            jitter: if not 0, adds small random offsets to the frame indices;
                this is useful so we don't always land on even or odd frames
            seed: random seed for jittering; if you set this to a fixed value,
                you probably want to set it only on the first video 
    """
    assert num_frames > 0

    capture = cv2.VideoCapture(path)
    frame_count = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    if frame_count <= 0: return None

    frame_idxs = np.linspace(0, frame_count - 1, num_frames, endpoint=True, dtype=np.int)
    if jitter > 0:
        np.random.seed(seed)
        jitter_offsets = np.random.randint(-jitter, jitter, len(frame_idxs))
        frame_idxs = np.clip(frame_idxs + jitter_offsets, 0, frame_count - 1)

    result = _read_frames_at_indices(path, capture, frame_idxs)
    capture.release()
    return result

def isotropically_resize_image(img, size, resample=cv2.INTER_AREA):
    h, w = img.shape[:2]
    if w > h:
        h = h * size // w
        w = size
    else:
        w = w * size // h
        h = size

    resized = cv2.resize(img, (w, h), interpolation=resample)
    return resized


def make_square_image(img):
    h, w = img.shape[:2]
    size = max(h, w)
    t = 0
    b = size - h
    l = 0
    r = size - w
    return cv2.copyMakeBorder(img, t, b, l, r, cv2.BORDER_CONSTANT, value=0)

In [5]:
sys.path.insert(1,'/content/drive/My Drive/YoloV2' )
import yolov2

mobilenetv2 = yolov2.load_mobilenetv2_224_075_detector("/content/drive/My Drive/YoloV2/facedetection-mobilenetv2-size224-alpha0.75.h5")
yolo_model = yolov2.FaceDetector(model=mobilenetv2)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
def get_faces(frames):
    faces = []
    for face in frames:
        yolo_boxes = yolo_model.detect(face, 0.7)
        yb = yolov2.get_boxes_points(yolo_boxes, face.shape)
        
        for b in yb:
            lx, ly, rx, ry = b
            # x, y, w, h here
            # ax.add_patch(Rectangle((lx,ly),rx - lx,ry - ly,linewidth=2,edgecolor='red',facecolor='none'))
            img_crop = face[ly:ry, lx:rx]
            resized_face = isotropically_resize_image(img_crop, input_size)
            resized_face = make_square_image(resized_face)
            faces.append(resized_face)
    return faces

# **Data Prep**

In [10]:
for path in tqdm(video_dir):
    if(os.path.exists(path[0]) == 0):
        print(path)

In [0]:
x_orig = []
y_orig = []
X_test = []
Y_test = []

In [12]:
for f in tqdm(video_dir):
    try:
        frames = read_frames(f[0])
        faces = get_faces(frames)
        x_orig.extend(faces)
        y_orig.extend([f[1]] * len(faces))
    except Exception as e:
        print("Error on video %s: %s" % (f[0], str(e)))

Error on video /content/drive/My Drive/Dataset/dfdc_train_part_19/uvpcmbutwe.mp4: 



In [0]:
x_np = np.array(x_orig, dtype="float32") / 255.0
y_np = np.array(y_orig)

In [14]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

(X_train, X_val, Y_train, Y_val) = train_test_split(x_np, y_np, test_size=0.25, random_state=42)

Y_train = to_categorical(Y_train, num_classes=2)
Y_val = to_categorical(Y_val, num_classes=2)

Using TensorFlow backend.


In [0]:
from keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

#LeNet Def

In [0]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as K

class LeNet:
	@staticmethod
	def build(width, height, depth, classes):
		# initialize the model
		model = Sequential()
		inputShape = (height, width, depth)
		# if we are using "channels first", update the input shape
		if K.image_data_format() == "channels_first":
			inputShape = (depth, height, width)
   
        # first set of CONV => RELU => POOL layers
		model.add(Conv2D(256, (5, 5), padding="same", input_shape=inputShape))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  
        # second set of CONV => RELU => POOL layers
		model.add(Conv2D(256, (5, 5), padding="same"))
		model.add(Activation("relu"))
		model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
  
        # first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(512))
		model.add(Activation("relu"))
		# softmax classifier
		model.add(Dense(classes))
		model.add(Activation("softmax"))
		# return the constructed network architecture
		return model

In [0]:
EPOCHS = 25
INIT_LR = 1e-3
BS = 32

In [0]:
from keras.optimizers import Adam

model = LeNet.build(width=224, height=224, depth=3, classes=2)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", 
              optimizer=opt,
	          metrics=["accuracy"])

In [25]:
H = model.fit_generator(aug.flow(X_train, Y_train, batch_size=BS),
	validation_data=(X_val, Y_val), steps_per_epoch=len(X_train) // BS,
	epochs=EPOCHS, verbose=1)

Epoch 1/25


ResourceExhaustedError: ignored

In [24]:
gc.collect()

538